In [2]:
%pylab inline
import numpy as np 

Populating the interactive namespace from numpy and matplotlib


# 기본적인 구조

의학 이미지안에서 질병등을 발견하는 것은 매우 중요합니다. <br>
Anomaly Detection GAN은 Discriminator를 통해서 의학 이미지 상에서 질병을 찾아내도록 도와줍니다. <br>
물론 Anomaly라는 개념하에서 반드시 질병일 필요는 없습니다. 


기본적인 구조는 아래와 같으며, Deep Convolutional Generative Adversarial Network (DCGAN)을 사용했으며, <br>
오른쪽 그림은 t-SNE embedding으로 파란색은 정상, 빨간색은 비정상(anomalous images) 를 나타냅니다.

<img src="./images/anogan.png" class="img-responsive img-rounded">

* Medical Images: $ \mathbf{I}_m $ 으로서 $ m = 1,2,..., M $ 이며, 모두 정상적인 상태의 (질병이 없는) 이미지, $ \mathbf{I}_m  \in \mathbb{R}^{a*b}$ 입니다.

**학습**시에는 오직 $ \mathbf{I}_m $ (질병없는 정상 이미지)가 주어지며, 해당 이미지로부터 c x c 크기의 image patch $ x_{k,m} $ 를 랜덤으로 위치를 샘플링해서 추출하게 됩니다.<br>
즉, $ \mathbf{x} = x_{k,m} \in \mathcal{X} $ 이며, 쉽게 설명하면 큰 메디컬 사진안에서 c x c 크기의 일부구간을 랜덤으로 잘라내고, 잘라진 구간을 비젼에서는 image patch 라고 말하며, 해당 image patch 는 질병이 없는 (위의 파란색) manifold  $ \mathcal{X} $ 를 학습한다는 뜻입니다. 

테스트시에는 $ \mathbf{y}_n, l_n $ 이 주어지며, $ \mathbf{y}_n $ 은 testing data $ \mathbf{J} $ 의 c x c 싸이즈의 이미지입니다.<br>
$ l_n \in \{0, 1\} $ 은 바이너리 값으로서 레이블값이라고 생각하면 되며, 오직 테스트를 할때만 정말 잘 되는지 판단하기 위해서 주어지는 값입니다.

## Minimax GAN

기본적인 GAN을 **복습**하는 차원에서 보도록 하겠습니다. (anomaly detection의 관점에서) <br>
GAN은 아시다시피 2개의 적대적인 모듈로 이루어 져 있습니다.<br>

* Discriminator $ G $ : $ G(\mathbf{z}) : \mathbf{z} \rightarrow \mathbf{x} $ 매핑을 통하여 정상적 데이터 (질병이 없는) $ \mathbf{x} $ 로 부터 distribution $ p_g $ 를 학습하며 1D -> 2D 이미지가 됩니다. <br>
* samples $ \mathbf{z} $ : 1차원의 vector라고 하면 되며, uniform distribution을 따르는 noise라고 생각하면 되며 Latent space $  \mathcal{Z} $ 로부터 샘플이 취해졌습니다.
* Image patch $ \mathbf{x} $ : 질병이 없는 정상적인 메디컬 사진에서 랜덤으로 c x c 싸이즈로 샘플링한 이미지이며, image space manifold $ \mathcal{X} $ 안에 속한다고 봅니다.

이러한 설정안에서, Generator $ G $ 는 convolutional decoder라고 보면 되며, 여러층을 쌓은 (stack) strided convolutions입니다.<br>
Discriminator $ D $ 는 일반적인 CNN이며 2D images를 받아서 scalar value $ D(\cdot) $ 으로 매핑합니다.<br>
결론적으로 Minimax game 을 통해 학습되며 Value function $ V(G, D) $ 는 다음과 같습니다.<br>

$$ \min_G \max_D V(D, G) =  
\mathbb{E}_{\mathbf{x} \sim P_{data}(\mathbf{x})} \big[ \log D(\mathbf{x}) \big] 
+ \mathbb{E}_{\mathbf{z} \sim P_{\mathbf{z}}(\mathbf{z})} \big[ \log \big( 1- d(G(\mathbf{z})) \big) \big] $$

Discriminator는 위의 공식으로 학습하면 되고, Generator는 minimax에 문제가 좀 있어서 아래와 같이 학습을 하게 됩니다.<br>
자세한 내용은 GAN을 문서를 확인합니다.

$$ V(G) = D(G(\mathbf{z})) $$ 

## Mapping new images to the Latent Space

GAN학습이 완료되면 Generator는 $ G(\mathbf{z}) = \mathbf{z} \rightarrow \mathbf{x} $ 처럼 latent space $ \mathbf{z} $ 로부터 정상적 메디컬 사진 (질병이 없는) 이미지 $ \mathbf{x} $ 로 매핑을 하게 됩니다.<br>
문제는 그 반대의 상황이 어렵습니다. 즉 $ \mu(\mathbf{x}) = \mathbf{x} \rightarrow \mathbf{z} $ 는 할 수가 없습니다.<br>


# Loss

## Residual Loss

Residual loss는 query image $ \mathbf{x} $ 와 생성된 이미지 $ G(\mathbf{z}_\gamma) $ 의 시각적 차이를 측정하기 위해서 사용이 됩니다.

$$ L_R \left(\mathbf{z}_{\gamma} \right) = \sum \big|\ \mathbf{x} - G(\mathbf{z}_\gamma) \ \big| $$ 



## Dataset

http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm